In [1]:
import cv2
import numpy as np
import math
import glob
from matplotlib import pyplot as plt
import random
from IPython.display import clear_output
import time
from sympy import Line, pi
import os

In [2]:
#Function to show an image and close the window on ESC
def show_img(image, title="image"):
    cv2.imshow(title, cv2.resize(image, None, fx=0.3, fy=0.3))
    k = cv2.waitKey(0) & 0xFF
    if k == 27:
        cv2.destroyAllWindows()

In [3]:
#Function to calculate distance between 2 points
def get_distance(p1, p2):
    return math.sqrt((p2.x-p1.x)**2 + (p2.y-p1.y)**2)

In [4]:
class Point:
    # x and y represent the pixel position in an image, rather than coordinates.
    # y = 0 means it would be on the first row of pixels of the image
    x = 0
    y = 0

    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def getCoordinates(self):
        return [self.x, self.y]
    def setX(self, x):
        self.x = x
    def setY(self, y):
        self.y = y

In [5]:
class Ellipse:
    x = 0
    y = 0
    minorAxis = 0
    majorAxis = 0

    def __init__(self, x, y, minorAxis, majorAxis):
        self.x = x
        self.y = y
        self.minorAxis = minorAxis
        self.majorAxis = majorAxis
    
    def getAxes(self):
        return([self.minorAxis, self.majorAxis])

In [6]:
def inside_ellipse(point, ellipse):
    eq = ((point.x - ellipse.x)**2)/ellipse.minorAxis**2 + ((point.y - ellipse.y)**2)/ellipse.majorAxis**2
    if eq < 1:
        return True
    return False

In [7]:
def warp_img_base(image, show_process=False, M=None):
    #Setting the 4 points used to warp the template board image
    top_point = Point(1260,340)
    left_point = Point(260,1540)
    bottom_point = Point(1330,2550)
    right_point = Point(1990,1450)

    height = math.ceil(get_distance(top_point,bottom_point))
    width = math.ceil(get_distance(left_point,right_point))

    input_pts = np.float32([top_point.getCoordinates(),left_point.getCoordinates(),bottom_point.getCoordinates(),right_point.getCoordinates()])
    output_pts = np.float32([[math.ceil(width/2), 0],[0, math.ceil(height/2)], [math.ceil(width/2), height-1], [width-1, math.ceil(height/2)]])

    image_copy = image.copy()
    if M is None:
        M = cv2.getPerspectiveTransform(input_pts, output_pts)
    warp = cv2.warpPerspective(image_copy, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)

    if show_process:
        show_img(warp, "warp")

    return warp

In [8]:
def warp_img_base2(image, show_process=False, M=None):
    #Setting the 4 points used to warp the template board image
    top_point = Point(1260,780)
    left_point = Point(324,1990)
    bottom_point = Point(1380,2900)
    right_point = Point(2018,1740)
    image_copy = image.copy()

    i = cv2.circle(image_copy, [top_point.x, top_point.y], radius=10, color=(25, 255, 0), thickness=-1)
    i = cv2.circle(image_copy, [left_point.x, left_point.y], radius=10, color=(25, 255, 0), thickness=-1)
    i = cv2.circle(image_copy, [bottom_point.x, bottom_point.y], radius=10, color=(25, 255, 0), thickness=-1)
    i = cv2.circle(image_copy, [right_point.x, right_point.y], radius=10, color=(25, 255, 0), thickness=-1)
    if show_process:
        show_img(image_copy)
    
    height = math.ceil(get_distance(top_point,bottom_point))
    width = math.ceil(get_distance(left_point,right_point))

    input_pts = np.float32([top_point.getCoordinates(),left_point.getCoordinates(),bottom_point.getCoordinates(),right_point.getCoordinates()])
    output_pts = np.float32([[math.ceil(width/2), 0],[0, math.ceil(height/2)], [math.ceil(width/2), height-1], [width-1, math.ceil(height/2)]])

    image_copy = image.copy()
    if M is None:
        M = cv2.getPerspectiveTransform(input_pts, output_pts)
    warp = cv2.warpPerspective(image_copy, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)

    if show_process:
        show_img(warp, "warp")

    origin = Point(860,1067)
    return warp, origin

In [9]:
def warp_img(image, img_board_alligned, show_process=False):
    # img_board_alligned = cv2.imread('./board_alligned.jpg')

    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_board_alligned = cv2.cvtColor(img_board_alligned, cv2.COLOR_BGR2GRAY)

    sift = cv2.SIFT_create()

    # Getting keypoints and descriptors
    keypoints_1, descriptors_1 = sift.detectAndCompute(image, None)
    keypoints_2, descriptors_2 = sift.detectAndCompute(img_board_alligned, None)

    # Feature matching
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(descriptors_1,descriptors_2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    img = cv2.drawMatches(image, keypoints_1, img_board_alligned, keypoints_2, good[:50], img_board_alligned, flags=2)
    if show_process:
        show_img(img, 'matches')

    src_pts = np.float32([ keypoints_1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ keypoints_2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()

    h,w = img_board_alligned.shape

    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)

    img_board_alligned = cv2.polylines(img_board_alligned,[np.int32(dst)],True,255,3, cv2.LINE_AA)

    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
    singlePointColor = None,
    matchesMask = matchesMask, # draw only inliers
    flags = 2)
    img3 = cv2.drawMatches(image,keypoints_1,img_board_alligned,keypoints_2,good,None,**draw_params)

    if show_process:
        show_img(img3, 'result')

    warp = warp_img_base(image, False,  M)
    if show_process:
        show_img(warp, 'warp')
    return warp

In [10]:
def get_dart_masks(idart, iboard, show_process=False):
    # iboard = cv2.imread('./board_alligned.jpg', cv2.IMREAD_GRAYSCALE)
    # iboard = cv2.cvtColor(idart, cv2.COLOR_BGR2GRAY)
    # idart = cv2.cvtColor(idart, cv2.COLOR_BGR2GRAY)

    iboard = cv2.GaussianBlur(iboard, (21,21), 0)
    idart = cv2.GaussianBlur(idart, (21,21), 0)

    diff = cv2.absdiff(idart, iboard)

    thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    # thresh = cv2.erode(thresh, np.ones((5,5), np.uint8), iterations=2)
    thresh = cv2.dilate(thresh, np.ones((15,15), np.uint8), iterations=10)
    thresh = cv2.erode(thresh, np.ones((5,5), np.uint8), iterations=30)
    masked = cv2.bitwise_and(idart, idart, mask=thresh)

    if show_process:
        show_img(thresh)
        show_img(masked)
    return thresh

In [11]:
def get_tips(dart_masks,show_process=False):
    analysis = cv2.connectedComponentsWithStats(dart_masks, connectivity=8)
    (totalLabels, label_ids, values, centroid) = analysis

    tips = []
    for i in range(1, totalLabels):
        area = values[i, cv2.CC_STAT_AREA]  
        
        if area > 5000:
            componentMask = (label_ids == i).astype("uint8")
            ys = []
            x = values[i, cv2.CC_STAT_LEFT]
            
            for y in range(0,componentMask.shape[0]):
                if componentMask[y, x] == 1:
                    ys.append(y)
            y = math.ceil(sum(ys) / len(ys))

            tip = Point(x, y)
            tips.append(tip)
            if show_process:
                componentMask = (label_ids == i).astype("uint8") * 255
                
                # Creating the Final output mask
                output = np.zeros(dart_masks.shape, dtype="uint8")
                output = cv2.bitwise_or(output, componentMask)
                show_img(output, str(i))
    return tips

In [12]:
def set_score_zones_task1(image, points=[], show_process=False):
    image_copy = image.copy()
    score_center = Point(866, 1115)
    # score_img = cv2.circle(image_copy, [score_center.x, score_center.y], radius=10, color=(25, 255, 0), thickness=-1)   

    pts_b = Ellipse(score_center.x + 2, score_center.y - 1, 30, 30)
    pts_9 = Ellipse(score_center.x - 6, score_center.y + 2, 126, 151)
    pts_8 = Ellipse(score_center.x - 6, score_center.y + 4, 225, 272)
    pts_7 = Ellipse(score_center.x - 5, score_center.y + 4, 305, 382)
    pts_6 = Ellipse(score_center.x - 7, score_center.y + 4, 404, 507)
    pts_5 = Ellipse(score_center.x - 5, score_center.y + 2, 485, 615)
    pts_4 = Ellipse(score_center.x - 5, score_center.y + 2, 585, 740)
    pts_3 = Ellipse(score_center.x - 5, score_center.y + 2, 660, 850)
    pts_2 = Ellipse(score_center.x - 5, score_center.y + 1, 770, 970)
    pts_1 = Ellipse(score_center.x, score_center.y + 1, 900, 1120)

    if show_process:
        score_img = cv2.ellipse(image_copy, [pts_b.x, pts_b.y], pts_b.getAxes(), 0, 0, 360, color=(152,50,0),thickness=4)
        score_img = cv2.ellipse(image_copy, [pts_9.x, pts_9.y], pts_9.getAxes(), 0, 0, 360, color=(152,50,0),thickness=4)
        score_img = cv2.ellipse(image_copy, [pts_8.x, pts_8.y], pts_8.getAxes(), 0, 0, 360, color=(152,50,0),thickness=4)
        score_img = cv2.ellipse(image_copy, [pts_7.x, pts_7.y], pts_7.getAxes(), 0, 0, 360, color=(152,50,0),thickness=4)
        score_img = cv2.ellipse(image_copy, [pts_6.x, pts_6.y], pts_6.getAxes(), 0, 0, 360, color=(152,50,0),thickness=4)
        score_img = cv2.ellipse(image_copy, [pts_5.x, pts_5.y], pts_5.getAxes(), 0, 0, 360, color=(152,50,0),thickness=4)
        score_img = cv2.ellipse(image_copy, [pts_4.x, pts_4.y], pts_4.getAxes(), 0, 0, 360, color=(152,50,0),thickness=4)
        score_img = cv2.ellipse(image_copy, [pts_3.x, pts_3.y], pts_3.getAxes(), 0, 0, 360, color=(152,50,0),thickness=4)
        score_img = cv2.ellipse(image_copy, [pts_2.x, pts_2.y], pts_2.getAxes(), 0, 0, 360, color=(152,50,0),thickness=4)
        score_img = cv2.ellipse(image_copy, [pts_1.x, pts_1.y], pts_1.getAxes(), 0, 0, 360, color=(152,50,0),thickness=4)
        for point in points:
            score_img = cv2.circle(image_copy, [point.x, point.y], radius=7, color=(25, 255, 0), thickness=-1)   
        show_img(image_copy)
    
    return ([pts_1, pts_2, pts_3, pts_4, pts_5, pts_6, pts_7, pts_8, pts_9, pts_b])

In [13]:
def calc_score_task1(tips, score_zones):
    scores = []
    for tip in tips:
        for i in reversed(range(1,len(score_zones))):
            if inside_ellipse(tip, score_zones[i]):
                print("Inside score zone " + str(i+1))
                scores.append(i+1)
                break
    return scores

In [14]:
def draw_points(points, image):
    image_copy = image.copy()
    for point in points:
        p = cv2.circle(image_copy, [point.x, point.y], radius=10, color=(25, 255, 0), thickness=-1)
    show_img(p)

In [15]:
def cart2polar(origin, point, height=None):
    if height is None:
        height = 3264
    x = point.x - origin.x
    y = (height - point.y) - (height - origin.y)
    rho = math.sqrt(x**2 + y**2)
    try:
        theta = math.atan(y/x)
    except:
        theta = math.atan(y/(x+1))
    theta = 180 * theta/math.pi

    if y >= 0 and x < 0:
        theta += 180
    if y < 0 and x < 0:
        theta = 180 + theta
    if y < 0 and x >= 0:
        theta = 360 + theta

    return rho, theta

In [16]:
def calc_score_zone(theta):
    #scores = ['error', 10, 15, 2, 17, 3, 19, 7, 16, 8, 11, 14, 9, 12, 5, 20, 1, 18, 4, 13]
    scores = ['error', 13, 4, 18, 1, 20, 5, 12, 9, 14, 11, 8, 16, 7, 19, 3, 17, 2, 15, 10]
    if theta >= 351 or theta < 9:
        return 6
    for i in range(9, 351, 18):
        if theta >= i and theta < i+18:
            return scores[(i+9)//18]

In [17]:
def calc_score_task2(origin, tips, img=None):
    scores = []
    # warp_copy = img.copy()
    for tip in tips:
        score = ''
        rho, theta = cart2polar(origin, tip)
        # print((rho, theta))
        if rho < 855:
            if rho < 30:
                score = 'b50'
            elif rho < 80:
                score = 'b25'
            elif rho > 470 and rho < 530:
                zone = calc_score_zone(theta)
                score = 't' + str(zone)
            elif rho > 785 and rho < 855:
                zone = calc_score_zone(theta)
                score = 'd' + str(zone)
            else:
                zone = calc_score_zone(theta)
                score = 's' + str(zone)
            
            scores.append(score)
    #     i = cv2.circle(warp_copy, [tip.x, tip.y], radius=10, color=(25, 255, 0), thickness=-1)
    # show_img(warp_copy)
        
    return scores

In [19]:
def calculate_grade(task):
    if task == 1:
        a_src = './Dataset/evaluation/fake_test/Task1/'
        r_src = './Dataset/train/Task1/'
    elif task == 2:
        a_src = './Dataset/evaluation/fake_test/Task2/'
        r_src = './Dataset/train/Task2/'
    else:
        raise Exception("Task no. invalid")
    
    no_arrows_guessed = 0
    pos_arrows_guessed = 0

    for i in range(1, 26):
        if(i < 10):
            answer_src = a_src + '0' + str(i) + '.txt'
            real_src = r_src  + '0' + str(i) + '.txt'
        else:
            answer_src = a_src + str(i) + '.txt'
            real_src = r_src  + str(i) + '.txt'

        answers = []
        real = []

        f = open(answer_src, 'r')
        for line in f.readlines():
            if line[-1] == '\n':
                line = line[:-1]
            answers.append(line)
        f.close()

        f = open(real_src, 'r')
        for line in f.readlines():
            if line[-1] == '\n':
                line = line[:-1]
            real.append(line)
        f.close()

        answers[-1] = answers[-1]
        
        if answers[0] == real[0]:
            no_arrows_guessed += 1
        answers = answers[1:]
        real = real[1:]

        j = 0
        while(j < len(answers)):
            if answers[j] in real:
                pos_arrows_guessed += 1
                real.remove(answers[j])
                answers.remove(answers[j])
                j -= 1
            j += 1
            
    if task == 1:
        grade = 0.04 * no_arrows_guessed + 0.02 * pos_arrows_guessed
        total = 2
    else:
        grade = 0.03 * no_arrows_guessed + 0.015 * pos_arrows_guessed
        total = 1.5
    print("Task: " + str(task) + '\n' + "Grade: " + str(grade) + "|" + str(total))
    return grade

In [522]:
calculate_grade(task=1)

Task: 1
Grade: 1.88|2


1.88

TASK 1

In [21]:
img_board_empty = cv2.imread('./auxiliary_images/template_task1.jpg')
try:
    os.mkdir('./test/Task1')
except:
    pass
for i in range(1, 26):
    img_source = './test/Task1/'
    if(i < 10):
        img_source = img_source + '0' + str(i) + '.jpg'
    else:
        img_source = img_source + str(i) + '.jpg'

    clear_output(wait=True)
    print("Image nr." + str(i))

    img_test = cv2.imread(img_source)

    img_board_alligned = warp_img_base(img_board_empty)
    img_test = warp_img(img_test, img_board_alligned, show_process=False)
    darts = get_dart_masks(img_test, img_board_alligned[:,:,0], show_process=False)
    tips = get_tips(darts, show_process=False)
    score_zones = set_score_zones_task1(img_board_alligned, points=tips, show_process=False)
    scores = calc_score_task1(tips, score_zones)

    filename = img_source[:-4] + '_predicted.' + 'txt'
    f = open(filename, 'w')
    f.write(str(len(scores))+'\n')
    for score in scores:
        f.write(str(score)+'\n')
    f.close()

Image nr.25
Inside score zone 5


TASK 2

In [22]:
img_board_empty = cv2.imread('./auxiliary_images/template_task2.jpg')
try:
    os.mkdir('./test/Task2')
except:
    pass
for i in range(1, 26):
    img_source = './test/Task2/'
    if(i < 10):
        img_source = img_source + '0' + str(i) + '.jpg'
    else:
        img_source = img_source + str(i) + '.jpg'

    clear_output(wait=True)
    print("Image nr." + str(i))

    img_test = cv2.imread(img_source)

    img_board_alligned, origin = warp_img_base2(img_board_empty)
    img_test = warp_img(img_test, img_board_alligned, show_process=False)
    darts = get_dart_masks(img_test, img_board_alligned[:,:,0], show_process=False)
    tips = get_tips(darts, show_process=False)
    scores = calc_score_task2(origin, tips, img_test)

    filename = img_source[:-4] + '_predicted.' + 'txt'
    f = open(filename, 'w')
    f.write(str(len(scores))+'\n')
    for score in scores:
        f.write(str(score)+'\n')
    f.close()


Image nr.25
